In [87]:
"""hamming_set.ipynb"""

# Cell 01

# We want all positive integers less than 256 with a Hamming
# weight of 2, 3, or 4. These three weights are mutually exclusive
# sets, so we can find the numbers separately
# 256 in binary is 100000000, meaning all positive integers less than
# that will have 8 digits in their bitstring

# I think the way I'm coming up with for this is building big
# matrices

'hamming_set.ipynb'

In [88]:
# Cell 02

import numpy as np


def arrange_single_bit(space: int):
    # OK this method is for arranging a single bit over a 'space'
    # For example, one bit over 7 open spaces
    # That's not very well explained, but hopefully it becomes clear

    # Create an empty list which will be the object holding a list of
    # the binary strings of all numbers with Hamming weight 2
    hamming_strings = []
    # Call a counter which starts with the same value as our space
    count = space

    # Start counting down!
    while count > 0:
        # First, we're going to concatenate a vector of ones to
        # an identity matrix of dimension count x count.
        # If count is 3 for example, we get 1100, 1010, 1001 in
        # 3 rows of a matrix
        place_holder = np.concatenate(
            (np.ones(count)[np.newaxis].T, np.identity(count)), axis=1
        )
        # We're looking for bit strings of length 8 to be consistent
        # See Cell 01 for 8 length rationale
        for _ in range(space - count):
            # So fill out any open space to the left with zeroes
            # From earlier example, 1100 -> 00001100
            place_holder = np.concatenate(
                (np.zeros(count)[np.newaxis].T, place_holder), axis=1
            )
        # For the very first pass, we want to replace the empty list
        # with the matrix of our values
        if count == space:
            hamming_strings = place_holder
        # For every subsequent pass, we want to put the next values
        # below the existing ones, extending our list
        else:
            hamming_strings = np.concatenate((hamming_strings, place_holder), axis=0)
        # Continue counting down
        count -= 1
    # Return the array of arranged bits!
    return hamming_strings


# Convert the array of bits to strings, join them, and base convert
def convert_to_decimal(strings_array):
    # Our matrices have shape (n, 8), but we really want (n, 1)
    # So we're going to change the values in the matrix to dtype int
    # then str, followed by joining the strings along each row
    hamming_strings = strings_array.astype(int).astype(str)
    hamming_strings = np.apply_along_axis("".join, 1, hamming_strings)
    # Once our strings are joined, we should have n bit strings
    # We're going to vectorize the python int function to
    # perform base conversion over the whole ndarray at once, since
    # our objective is to collect all of the integers by their hamming weight
    base_converter = np.vectorize(int)
    return base_converter(hamming_strings, 2)


In [89]:
# Cell 03

# Consider the case of hamming weight of 2. This is just arranging
# 2 ones over the space of 8 open bit positions.
# And that problem is equivalent to fixing the first at position 1,
# then arranging the second over 7 positions. Then fixing bit 1 at space 2
# and arranging one bit over 6 positions. Repeat, repeat, until one bit
# is arranged over 7, 6, 5, 4, 3, 2, 1 spaces.

# In the function in Cell 02, we do that arranging. We produce
# all of those combinations, then go back to decimal integers
# So, if you want to know all of the integers less than 256 with
# hamming weight 2, that is just...


print("The integers less than 256 with Hamming weight 2 are: ")
print(np.sort(hamming_two := convert_to_decimal(arrange_single_bit(7))))

The integers less than 256 with Hamming weight 2 are: 
[  3   5   6   9  10  12  17  18  20  24  33  34  36  40  48  65  66  68
  72  80  96 129 130 132 136 144 160 192]


In [90]:
# Cell 04

# To fins all the integers of Hamming weight 3, we can take a
# very similar approach. If we fix the position of the first,
# the problem becomes arranging 2 integers over 7 spaces, which
# is actually arranging one over 6, 5, 4, 3, 2, 1 spaces.
# Then when we move the position of the first one bit, it's arranging
# two bits over 6 spaces, which is actually one over 5, 4, 3, 2, 1
# This goes on, and we can say the problem is really just arranging
# two bits over 7, 6, 5, 4, 3, 2 spaces

# So let's write a function that can do that arranging

# Start out by specifying the number of spaces that you want to
# arrange the two bits over.
# For the case of hamming weight 3, we arrange 2 bits over 7 spaces
# to start
def arrange_two_bits(spaces):
    new_hamming_strings = []
    # Start the count
    count = spaces
    # Since we can't arrange 2 bits on 1 space, we need to stop earlier
    while count > 1:
        # Same principle as the first function, we're going to concatenate
        # a vector of ones in front of an identity matrix
        # But this time, we're arranging the one bit over 6, 5, 4, 3, 2, 1
        # spaces, so we need a vector of ones with length 21
        # But then we're going to repeat, and we'll need a vector of length 15
        # And then on and on
        place_holder = np.concatenate(
            (
                # See above comment for the rationale behind sum(range(count))
                np.ones(sum(range(count)))[np.newaxis].T,
                # And we're arranging the single bit over a space 1 smaller
                arrange_single_bit(count - 1),
            ),
            axis=1,
        )
        # We're looking for bit strings of length 8 to be consistent
        # See Cell 01 for 8 length rationale
        for _ in range(spaces - count):
            # So fill out any open space to the left with zeroes
            place_holder = np.concatenate(
                (np.zeros(sum(range(count)))[np.newaxis].T, place_holder), axis=1
            )
        # For the very first pass, we want to replace the empty list
        # with the matrix of our values
        if count == spaces:
            new_hamming_strings = place_holder
        # For every subsequent pass, we want to put the next values
        # below the existing ones, extending our list
        else:
            new_hamming_strings = np.concatenate(
                (new_hamming_strings, place_holder), axis=0
            )
        # Continue counting down
        count -= 1
    return new_hamming_strings

In [91]:
# Cell 05

# Same principle as before, we can get those integers with hamming
# weight of 3 by reducing the problem to arranging 2 bits, then one
# bit.

print("The integers less than 256 with Hamming weight 3 are: ")
print(np.sort(hamming_three := convert_to_decimal(arrange_two_bits(7))))

The integers less than 256 with Hamming weight 3 are: 
[  7  11  13  14  19  21  22  25  26  28  35  37  38  41  42  44  49  50
  52  56  67  69  70  73  74  76  81  82  84  88  97  98 100 104 112 131
 133 134 137 138 140 145 146 148 152 161 162 164 168 176 193 194 196 200
 208 224]


In [92]:
# Cell 06

# Finally, we have to find those with Hamming Weight 4
# This is just like those with 2 and 3, we're going to reduce the problem
# 4 ones over 8 spaces is 3 over 7 which is 2 over 6, which is one over 5
# The problem is growing a bit out of proportions but we'll stick
# to our trick


def arrange_three_bits(spaces: int):
    new_hamming_strings = []
    count = spaces

    # Once again, we can't arrange 3 bits over 2 spaces, so we stop early
    while count > 2:
        place_holder = np.concatenate(
            (
                # This time, we're arranging more and more matrices, so we need to take
                # the cumsum, them sum the cumsums!
                np.ones(sum(np.cumsum(range(count - 1))))[np.newaxis].T,
                # And we're arranging the two bits over a space 1 smaller
                arrange_two_bits(count - 1),
            ),
            axis=1,
        )
        # We're looking for bit strings of length 8 to be consistent
        # See Cell 01 for 8 length rationale
        for _ in range(spaces - count):
            # So fill out any open space to the left with zeroes
            place_holder = np.concatenate(
                (
                    np.zeros(sum(np.cumsum(range(count - 1))))[np.newaxis].T,
                    place_holder,
                ),
                axis=1,
            )
        # For the very first pass, we want to replace the empty list
        # with the matrix of our values
        if count == spaces:
            new_hamming_strings = place_holder
        # For every subsequent pass, we want to put the next values
        # below the existing ones, extending our list
        else:
            new_hamming_strings = np.concatenate(
                (new_hamming_strings, place_holder), axis=0
            )
        # Continue counting down
        count -= 1
    return new_hamming_strings

In [93]:
# Cell 07

# Same idea as before, now we want to print out the integers
# with hamming weight 4, so we employ the problem of arranging 3 bits over
# 7, 6, 5, 4, 3 spaces...

print("The integers less than 256 with Hamming weight 4 are: ")
print(np.sort(hamming_four := convert_to_decimal(arrange_three_bits(7))))

The integers less than 256 with Hamming weight 4 are: 
[ 15  23  27  29  30  39  43  45  46  51  53  54  57  58  60  71  75  77
  78  83  85  86  89  90  92  99 101 102 105 106 108 113 114 116 120 135
 139 141 142 147 149 150 153 154 156 163 165 166 169 170 172 177 178 180
 184 195 197 198 201 202 204 209 210 212 216 225 226 228 232 240]


In [94]:
# Cell 08

# To display all positive integers less than 256 with Hamming
# weights 2, 3, or 4, we can concatenate these matrices and sort

all_integers = np.concatenate((hamming_two, hamming_three, hamming_four))

print("All of the integers less than 256 with Hamming weight ", end="")
print(f"of 2, 3, or 4: \n {np.sort(all_integers)}")

All of the integers less than 256 with Hamming weight of 2, 3, or 4: 
 [  3   5   6   7   9  10  11  12  13  14  15  17  18  19  20  21  22  23
  24  25  26  27  28  29  30  33  34  35  36  37  38  39  40  41  42  43
  44  45  46  48  49  50  51  52  53  54  56  57  58  60  65  66  67  68
  69  70  71  72  73  74  75  76  77  78  80  81  82  83  84  85  86  88
  89  90  92  96  97  98  99 100 101 102 104 105 106 108 112 113 114 116
 120 129 130 131 132 133 134 135 136 137 138 139 140 141 142 144 145 146
 147 148 149 150 152 153 154 156 160 161 162 163 164 165 166 168 169 170
 172 176 177 178 180 184 192 193 194 195 196 197 198 200 201 202 204 208
 209 210 212 216 224 225 226 228 232 240]


In [95]:
# While this isn't really a viable algorithm for getting to larger
# and larger hamming weights, it's at least a start and an idea
# that plays on matrices in a fun way maybe